In [1]:
import os
import json
import ray
import numpy as np
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
from flow.networks import Network
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, VehicleParams, SumoCarFollowingParams, InFlows, SumoLaneChangeParams
from flow.controllers import RLController, IDMController, SimLaneChangeController
from gym.spaces.box import Box
from gym.spaces import Tuple

from constructionRouter import ConstructionRouter
from constructionEnvV2 import myEnv

# time horizon of a single rollout
HORIZON = 5000
# number of rollouts per training iteration
N_ROLLOUTS = 10
# number of parallel workers
N_CPUS = 2

vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
             ),
             num_vehicles=10
             )
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode=25
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode=1621),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["edge4"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "edge1": ["edge1","edge2","edge3","edge4","edge5","edge6"],
                #"edge3": ["edge3","edge4","edge5","edge10","edge11","edge12","edge3"],
                "edge4": ["edge4","edge5","edge10","edge11","edge12","edge3","edge4"],
                }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="edge1",
           vehs_per_hour=10000,
            depart_lane="random",
            depart_speed="random",
            color="white")

file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/construction/constructionV5.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.5,
        render=False,
        restart_instance=True,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=1000,
        clip_actions=False,
        additional_params={
            "target_velocity": 50,
            "sort_vehicles": False,
            "max_accel": 2,
            "max_decel": 2,
            "lane_change_duration": 5,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [32, 32, 32, 32]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config


/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:

alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 5,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})

2020-04-06 08:13:26,449	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-04-06 08:13:26,452	INFO resource_spec.py:212 -- Starting Ray with 4.83 GiB memory available for workers and up to 2.42 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-06 08:13:26,818	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_057338e0,RUNNING,


(pid=raylet) E0406 08:13:26.978584800    1789 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1586175206.978566500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=raylet) E0406 08:13:26.978842400    1789 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E0406 08:13:26.996826100    1789 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=1813) E0406 08:13:28.111664300    1813 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1586175208.111652900","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=1

(pid=1813) 2020-04-06 08:13:41,453	WARNING util.py:41 -- Install gputil for GPU system monitoring.
(pid=1811) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=1811)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=1811) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=1811)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=1811) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be un

(pid=1812) Warning: Vehicle 'flow_00.115' performs emergency braking with decel=-13.50 wished=4.50 severity=2.00, time=146.00.
(pid=1812) Warning: Vehicle 'flow_00.106' performs emergency braking with decel=-13.34 wished=4.50 severity=1.96, time=146.50.
(pid=1812) Warning: Vehicle 'flow_00.110' performs emergency braking with decel=-14.77 wished=4.50 severity=2.28, time=147.00.
(pid=1812) Warning: Vehicle 'flow_00.117' performs emergency braking with decel=-15.42 wished=4.50 severity=2.43, time=147.50.
(pid=1812) Warning: Vehicle 'flow_00.124' performs emergency braking with decel=-12.12 wished=4.50 severity=1.69, time=148.00.
(pid=1811) Warning: Vehicle 'rl_6' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=147.50.
(pid=1811) Warning: Vehicle 'flow_00.102' performs emergency braking with decel=-10.99 wished=4.50 severity=1.44, time=149.50.
(pid=1811) Warning: Vehicle 'flow_00.43' performs emergency braking with decel=-21.35 wished=4.50 severity=3.74, time=1

(pid=1812) Warning: Vehicle 'flow_00.582' performs emergency braking with decel=-13.06 wished=4.50 severity=1.90, time=571.50.
(pid=1812) Warning: Vehicle 'flow_00.589' performs emergency braking with decel=-12.43 wished=4.50 severity=1.76, time=572.00.
(pid=1812) Warning: Vehicle 'flow_00.584' performs emergency braking with decel=-11.81 wished=4.50 severity=1.63, time=572.50.
(pid=1812) Warning: Vehicle 'flow_00.280' performs emergency braking with decel=-11.31 wished=4.50 severity=1.51, time=573.00.
(pid=1812) Warning: Vehicle 'flow_00.585' performs emergency braking with decel=-9.37 wished=4.50 severity=1.08, time=573.50.
(pid=1812) Warning: Vehicle 'flow_00.594' performs emergency braking with decel=-9.47 wished=4.50 severity=1.10, time=574.00.
(pid=1811) Warning: Vehicle 'flow_00.528' performs emergency braking with decel=-11.20 wished=4.50 severity=1.49, time=568.50.
(pid=1811) Warning: Vehicle 'flow_00.526' performs emergency braking with decel=-12.73 wished=4.50 severity=1.83,

(pid=1811) Warning: Vehicle 'flow_00.381' performs emergency braking with decel=-10.67 wished=4.50 severity=1.37, time=437.50.
(pid=1811) Warning: Vehicle 'flow_00.432' performs emergency braking with decel=-13.07 wished=4.50 severity=1.91, time=482.50.
(pid=1811) /mnt/c/Users/llave/Documents/GitHub/flow/flow/core/kernel/vehicle/traci.py:957: UserWarning: API change now handles duration as floating point seconds
(pid=1811)   veh_id, int(target_lane), 100000)
(pid=1811) Warning: Vehicle 'flow_00.458' performs emergency braking with decel=-13.72 wished=4.50 severity=2.05, time=506.00.
(pid=1812) Warning: Vehicle 'rl_9' performs emergency braking with decel=-14.18 wished=4.50 severity=2.15, time=961.50.
(pid=1811) Warning: Vehicle 'flow_00.560' performs emergency braking with decel=-15.29 wished=4.50 severity=2.40, time=555.00.
(pid=1811) Warning: Vehicle 'flow_00.469' performs emergency braking with decel=-24.61 wished=4.50 severity=4.47, time=601.50.
(pid=1811) Warning: Vehicle 'flow_00

(pid=1811) Warning: Vehicle 'flow_00.249' performs emergency braking with decel=-12.63 wished=4.50 severity=1.81, time=320.00.
(pid=1811) Warning: Vehicle 'flow_00.293' performs emergency braking with decel=-11.95 wished=4.50 severity=1.66, time=344.50.
(pid=1811) Warning: Vehicle 'flow_00.317' performs emergency braking with decel=-11.18 wished=4.50 severity=1.48, time=371.50.
(pid=1812) Warning: Vehicle 'flow_00.326' performs emergency braking with decel=-18.60 wished=4.50 severity=3.13, time=359.00.
(pid=1812) Warning: Vehicle 'flow_00.312' performs emergency braking with decel=-14.49 wished=4.50 severity=2.22, time=359.50.
(pid=1811) Warning: Vehicle 'flow_00.329' performs emergency braking with decel=-12.75 wished=4.50 severity=1.83, time=383.50.
(pid=1812) Warning: Vehicle 'flow_00.328' performs emergency braking with decel=-14.32 wished=4.50 severity=2.18, time=372.50.
(pid=1812) Warning: Vehicle 'flow_00.334' performs emergency braking with decel=-13.20 wished=4.50 severity=1.9

(pid=1812) Warning: Vehicle 'rl_6' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=137.00.
